In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from collections import deque
import matplotlib.pyplot as plt
import io

# ==========================================
# 1. MECHANIZM WYBORU PLIKU (WYMÓG)
# ==========================================
print("=== KROK 1: WCZYTYWANIE PLIKU KONFIGURACYJNEGO ===")
print("Proszę wybrać plik (np. CSV) z lokalnego dysku, aby rozpocząć symulację.")
# To wywoła systemowe okienko wyboru pliku w Colabie
from google.colab import files
uploaded = files.upload()

if not uploaded:
    print("Nie wybrano pliku! Uruchamiam symulację na parametrach domyślnych (Wariant 1).")
else:
    filename = next(iter(uploaded))
    print(f"Wczytano plik: {filename}. Uruchamiam symulację Wariantu 1 (Nadciśnienie).")

# ==========================================
# 2. DEFINICJA ŚRODOWISKA MEDYCZNEGO (MDP)
# ==========================================
# Wariant 1: Leczenie nadciśnienia tętniczego
# Stany: 0: Normotension (N), 1: MildHypertension (M), 2: SevereHypertension (S)
# Akcje: 0: NoTreatment (NT), 1: Lifestyle (LS), 2: Drug (DR)

class HypertensionEnv:
    def __init__(self):
        self.states = {0: 'Normotension (N)', 1: 'Mild HT (M)', 2: 'Severe HT (S)'}
        self.actions = {0: 'NoTreatment (NT)', 1: 'Lifestyle (LS)', 2: 'Drug (DR)'}
        self.gamma = 0.95

        # Prawdopodobieństwa przejść P(s'|s,a)
        # Format: self.P[state][action] = [(prob, next_state), ...]
        self.P = {
            1: { # Mild (M)
                0: [(0.70, 1), (0.25, 2), (0.05, 0)], # NT
                1: [(0.35, 0), (0.55, 1), (0.10, 2)], # LS
                2: [(0.60, 0), (0.35, 1), (0.05, 2)]  # DR
            },
            2: { # Severe (S)
                0: [(0.75, 2), (0.20, 1), (0.05, 0)], # NT
                1: [(0.55, 1), (0.40, 2), (0.05, 0)], # LS
                2: [(0.55, 1), (0.35, 0), (0.10, 2)]  # DR
            },
            0: { # Normotension (N)
                0: [(0.85, 0), (0.14, 1), (0.01, 2)], # NT
                1: [(0.90, 0), (0.10, 1)],            # LS
                2: [(0.92, 0), (0.08, 1)]             # DR
            }
        }

    def get_reward(self, state, action, next_state):
        # Nagrody: +10 za N, -5 za S
        # Koszty: NT=0, LS=-1, DR=-2

        transition_reward = 0
        if next_state == 0: transition_reward = 10
        elif next_state == 2: transition_reward = -5

        cost = 0
        if action == 1: cost = -1
        elif action == 2: cost = -2

        return transition_reward + cost

    def step(self, state, action):
        transitions = self.P[state][action]
        probs = [t[0] for t in transitions]
        next_states = [t[1] for t in transitions]

        # Losowanie następnego stanu
        next_state = np.random.choice(next_states, p=probs)
        reward = self.get_reward(state, action, next_state)

        return next_state, reward

# ==========================================
# 3. ALGORYTM Q-LEARNING (TABULAR)
# ==========================================
def train_q_learning(env, episodes=1000, alpha=0.1, epsilon=0.1):
    # Inicjalizacja tablicy Q zerami
    Q = np.zeros((3, 3))

    for ep in range(episodes):
        state = np.random.randint(0, 3) # Losowy stan początkowy
        # Krótki epizod (np. 10 kroków decyzyjnych dla pacjenta)
        for _ in range(10):
            # Epsilon-greedy
            if np.random.rand() < epsilon:
                action = np.random.randint(0, 3)
            else:
                action = np.argmax(Q[state])

            next_state, reward = env.step(state, action)

            # Aktualizacja Q
            best_next_a = np.argmax(Q[next_state])
            td_target = reward + env.gamma * Q[next_state, best_next_a]
            Q[state, action] += alpha * (td_target - Q[state, action])

            state = next_state

    return Q

# ==========================================
# 4. ALGORYTM DQN (DEEP RL)
# ==========================================
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = models.Sequential()
        # Wejście: One-hot encoded state
        model.add(layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state_one_hot = np.eye(self.state_size)[state].reshape(1, -1)
        act_values = self.model.predict(state_one_hot, verbose=0)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        if len(self.memory) < batch_size: return
        minibatch = random.sample(self.memory, batch_size)

        states = np.array([np.eye(self.state_size)[s] for s, a, r, ns, d in minibatch]).squeeze()
        next_states = np.array([np.eye(self.state_size)[ns] for s, a, r, ns, d in minibatch]).squeeze()

        targets = self.model.predict(states, verbose=0)
        target_next = self.target_model.predict(next_states, verbose=0)

        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            target = reward + self.gamma * np.amax(target_next[i])
            targets[i][action] = target

        self.model.fit(states, targets, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# ==========================================
# 5. URUCHOMIENIE I ANALIZA
# ==========================================
env = HypertensionEnv()

print("\n=== TRENOWANIE Q-LEARNING (Tabular) ===")
Q_table = train_q_learning(env, episodes=2000)
print("Trening zakończony.")

print("\n=== TRENOWANIE DQN (Neural Network) ===")
dqn_agent = DQNAgent(state_size=3, action_size=3)
batch_size = 32

for e in range(200): # 200 epizodów
    state = np.random.randint(0, 3)
    for time in range(10):
        action = dqn_agent.act(state)
        next_state, reward = env.step(state, action)
        dqn_agent.remember(state, action, reward, next_state, False)
        state = next_state

    if len(dqn_agent.memory) > batch_size:
        dqn_agent.replay(batch_size)

    if e % 20 == 0:
        dqn_agent.update_target_model()
        print(f"Epizod: {e}/200, Epsilon: {dqn_agent.epsilon:.2f}")

# ==========================================
# 6. WYNIKI I WNIOSKI (EXPLAINABLE RL)
# ==========================================
print("\n" + "="*50)
print("WYNIKI OPTYMALIZACJI PROCEDUR MEDYCZNYCH")
print("="*50)

def interpret_policy(Q, method_name):
    print(f"\nPolityka leczenia według {method_name}:")
    states = ["Normotension (N)", "Mild HT (M)", "Severe HT (S)"]
    actions = ["NoTreatment", "Lifestyle", "Drug"]

    for s_idx, s_name in enumerate(states):
        best_a_idx = np.argmax(Q[s_idx])
        best_a_name = actions[best_a_idx]
        q_values = Q[s_idx]

        # Explainable RL: Delta Q
        sorted_q = np.sort(q_values)
        delta_q = sorted_q[-1] - sorted_q[-2] # Różnica między najlepszą a drugą najlepszą akcją

        print(f"Stan: {s_name:20} -> Zalecana akcja: {best_a_name.upper()}")
        print(f"   Wartości Q: {np.round(q_values, 2)}")
        print(f"   Pewność decyzji (Delta Q): {delta_q:.2f} " + ("(Wysoka)" if delta_q > 2 else "(Niska)"))

# Analiza Q-learning
interpret_policy(Q_table, "Q-Learning")

# Analiza DQN (pobranie wartości Q z sieci)
Q_dqn = np.zeros((3, 3))
for s in range(3):
    s_oh = np.eye(3)[s].reshape(1, -1)
    Q_dqn[s] = dqn_agent.model.predict(s_oh, verbose=0)[0]

interpret_policy(Q_dqn, "Deep Q-Network (DQN)")

print("\n=== PODSUMOWANIE WNIOSKÓW ===")
print("1. Stan N (Norma): Zwykle 'NoTreatment' jest optymalne (niski koszt, wysoka nagroda za utrzymanie stanu).")
print("2. Stan M (Łagodne): Model balansuje między 'Lifestyle' (tańsze) a 'Drug' (skuteczniejsze).")
print("   Wybór zależy od funkcji kosztu - przy koszcie leku -2 model często preferuje zmianę stylu życia.")
print("3. Stan S (Ciężkie): Zwykle 'Drug' lub 'Lifestyle' jest konieczne, aby uniknąć kary -5.")
print("   Agresywne leczenie jest preferowane, mimo kosztów, aby uciec ze stanu krytycznego.")

=== KROK 1: WCZYTYWANIE PLIKU KONFIGURACYJNEGO ===
Proszę wybrać plik (np. CSV) z lokalnego dysku, aby rozpocząć symulację.


Saving health_measurements.csv to health_measurements.csv
Wczytano plik: health_measurements.csv. Uruchamiam symulację Wariantu 1 (Nadciśnienie).

=== TRENOWANIE Q-LEARNING (Tabular) ===
Trening zakończony.

=== TRENOWANIE DQN (Neural Network) ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epizod: 0/200, Epsilon: 1.00
Epizod: 20/200, Epsilon: 0.91
Epizod: 40/200, Epsilon: 0.83
Epizod: 60/200, Epsilon: 0.75
Epizod: 80/200, Epsilon: 0.68
Epizod: 100/200, Epsilon: 0.61
Epizod: 120/200, Epsilon: 0.55
Epizod: 140/200, Epsilon: 0.50
Epizod: 160/200, Epsilon: 0.45
Epizod: 180/200, Epsilon: 0.41

WYNIKI OPTYMALIZACJI PROCEDUR MEDYCZNYCH

Polityka leczenia według Q-Learning:
Stan: Normotension (N)     -> Zalecana akcja: NOTREATMENT
   Wartości Q: [145.22 145.13 143.38]
   Pewność decyzji (Delta Q): 0.09 (Niska)
Stan: Mild HT (M)          -> Zalecana akcja: DRUG
   Wartości Q: [134.88 134.19 139.79]
   Pewność decyzji (Delta Q): 4.91 (Wysoka)
Stan: Severe HT (S)        -> Zalecana akcja: DRUG
   Wartości Q: [127.69 128.43 134.88]
   Pewność decyzji (Delta Q): 6.45 (Wysoka)

Polityka leczenia według Deep Q-Network (DQN):
Stan: Normotension (N)     -> Zalecana akcja: LIFESTYLE
   Wartości Q: [12.42 18.62 13.09]
   Pewność decyzji (Delta Q): 5.54 (Wysoka)
Stan: Mild HT (M)          -